In [1]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import os
folder_path = '/content/drive/MyDrive/Part Sem Project'
os.chdir(folder_path)

Mounted at /content/drive


In [3]:
file_name = 'Disaster_Train.xlsx'
file_name2 = 'Disaster_Test.xlsx'
file_name3 ='Disaster_Dev.xlsx'
data_train = pd.read_excel(file_name, engine='openpyxl')
data_dev=pd.read_excel(file_name3, engine='openpyxl')
data_test = pd.read_excel(file_name2, engine='openpyxl')

In [4]:

data1 = pd.concat([data_train, data_test], ignore_index=True)
data1 = pd.concat([data1, data_dev], ignore_index=True)

data1.head()

,tweet_id,tweet_text,class_label,year,type,place
0,735891446960622976,RT @DonBradshawNTV: How @MarshallAmpsUK came t...,other_relevant_information,2016,wildfires,Canada
1,731202020296818048,Red Cross distributes $30M to Fort McMurray wi...,displaced_people_and_evacuations,2016,wildfires,Canada
2,733665357236342016,Interesting insights on the shifting communica...,other_relevant_information,2016,wildfires,Canada
3,731963038429928960,RT @globeandmail: Oil sands producers helping ...,rescue_volunteering_or_donation_effort,2016,wildfires,Canada
4,728674838034944000,Ottawa to match Red Cross donations for Fort M...,rescue_volunteering_or_donation_effort,2016,wildfires,Canada


In [5]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76173 entries, 0 to 76172
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   tweet_id     76173 non-null  int64 
 1   tweet_text   76173 non-null  object
 2   class_label  76173 non-null  object
 3   year         76173 non-null  int64 
 4   type         76173 non-null  object
 5   place        76173 non-null  object
dtypes: int64(2), object(4)
memory usage: 3.5+ MB


In [6]:
cnt = data1.groupby(['class_label']).nunique()
print(cnt)

                                        tweet_id  tweet_text  year  type  \
class_label                                                                
caution_and_advice                          5378        5378     4     5   
displaced_people_and_evacuations            3941        3941     4     5   
infrastructure_and_utility_damage           8133        8132     4     5   
injured_or_dead_people                      7303        7303     4     5   
missing_or_found_people                      358         358     2     3   
not_humanitarian                            6281        6281     4     5   
other_relevant_information                 12104       12104     4     5   
requests_or_urgent_needs                    2615        2615     4     5   
rescue_volunteering_or_donation_effort     21152       21150     4     5   
sympathy_and_support                        8908        8906     4     5   

                                        place  
class_label                            

In [7]:
class_labels = data1['class_label'].unique()
class_labels

array(['other_relevant_information', 'displaced_people_and_evacuations',
       'rescue_volunteering_or_donation_effort', 'caution_and_advice',
       'sympathy_and_support', 'infrastructure_and_utility_damage',
       'not_humanitarian', 'requests_or_urgent_needs',
       'injured_or_dead_people', 'missing_or_found_people'], dtype=object)

In [8]:
df=data1.copy()
df.head()

,tweet_id,tweet_text,class_label,year,type,place
0,735891446960622976,RT @DonBradshawNTV: How @MarshallAmpsUK came t...,other_relevant_information,2016,wildfires,Canada
1,731202020296818048,Red Cross distributes $30M to Fort McMurray wi...,displaced_people_and_evacuations,2016,wildfires,Canada
2,733665357236342016,Interesting insights on the shifting communica...,other_relevant_information,2016,wildfires,Canada
3,731963038429928960,RT @globeandmail: Oil sands producers helping ...,rescue_volunteering_or_donation_effort,2016,wildfires,Canada
4,728674838034944000,Ottawa to match Red Cross donations for Fort M...,rescue_volunteering_or_donation_effort,2016,wildfires,Canada


In [9]:
df = df.drop(df[df['class_label'] == 'class_label'].index)

class_labels = df['class_label'].unique()
class_labels

array(['other_relevant_information', 'displaced_people_and_evacuations',
       'rescue_volunteering_or_donation_effort', 'caution_and_advice',
       'sympathy_and_support', 'infrastructure_and_utility_damage',
       'not_humanitarian', 'requests_or_urgent_needs',
       'injured_or_dead_people', 'missing_or_found_people'], dtype=object)

In [10]:
#data2 = data2[pd.notnull(df['tags'])]
#print(data2.head(10))
print(df['tweet_text'].apply(lambda x: len(x.split(' '))).sum())

1694767


## Data Preprocessing

In [11]:
# Cleaning Special Characters and Removing Punctuation

import re
def clean_text(x):
    pattern = r'[^a-zA-z0-9\s]'
    text = re.sub(pattern, '', x)
    return x


In [12]:
#Cleaning Numbers

def clean_numbers(x):
    if bool(re.search(r'\d', x)):
        x = re.sub('[0-9]{5,}', '#####', x)
        x = re.sub('[0-9]**{4}**', '####', x)
        x = re.sub('[0-9]**{3}**', '###', x)
        x = re.sub('[0-9]**{2}**', '##', x)
    return x


In [13]:
#REMOVING CONTRACTIONS
contraction_dict = {"ain't": "is not", "aren't": "are not", "can't": "cannot", "'cause": "because", "could've": "could have","didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "I'd": "I would",
    "I'll": "I will",
    "I'm": "I am"}

def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re

contractions, contractions_re = _get_contractions(contraction_dict)

def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

# Usage
result = replace_contractions("this's a text with contraction")
print(result)


this's a text with contraction


## Data Representation: Sequence Creation

In [14]:
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

# Define the maximum number of words to consider (adjust as needed)
max_features = 1600000

# Split the dataset into training and testing sets (assuming an 80/20 split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Extract the 'tweet_text' and 'class_label' columns
train_X = train_df['tweet_text'].values
test_X = test_df['tweet_text'].values
train_y = train_df['class_label'].values
test_y = test_df['class_label'].values

In [15]:
#TOKENIZER

from keras.preprocessing.text import Tokenizer
## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(train_X)+list(test_X))
train_X = tokenizer.texts_to_sequences(train_X)
test_X = tokenizer.texts_to_sequences(test_X)

In [16]:
from keras.preprocessing.sequence import pad_sequences  # Import pad_sequences

maxlen=100
train_X = pad_sequences(train_X, maxlen=maxlen)
test_X = pad_sequences(test_X, maxlen=maxlen)

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
train_y = le.fit_transform(train_y)
test_y = le.transform(test_y)


## Load Embedding

In [19]:
def load_glove(word_index):
    EMBEDDING_FILE = 'glove.840B.300d.txt'

    def get_coefs(word, *arr):
        return word, np.asarray(arr, dtype='float32')[:300]

    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE))

    all_embs = np.stack(list(embeddings_index.values()))  # Convert to list
    emb_mean, emb_std = -0.005838499, 0.48782197
    embed_size = all_embs.shape[1]

    nb_words = min(max_features, len(word_index) + 1)
    embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

    for word, i in word_index.items():
        if i >= max_features:
            continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
        else:
            embedding_vector = embeddings_index.get(word.capitalize())
            if embedding_vector is not None:
                embedding_matrix[i] = embedding_vector

    return embedding_matrix

embedding_matrix = load_glove(tokenizer.word_index)


# CNN

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN_Text(nn.Module):
    def __init__(self):
        super(CNN_Text, self).__init__()
        filter_sizes = [1, 2, 3, 5]
        num_filters = 36
        n_classes = len(le.classes_)
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.convs1 = nn.ModuleList([nn.Conv2d(1, num_filters, (K, embed_size)) for K in filter_sizes])
        self.dropout = nn.Dropout(0.1)
        self.fc1 = nn.Linear(len(filter_sizes) * num_filters, n_classes)

    def forward(self, x):
        x = self.embedding(x)
        x = x.unsqueeze(1)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]
        x = torch.cat(x, 1)
        x = self.dropout(x)
        logit = self.fc1(x)
        return logit


In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define the model
model = torch.nn.Linear(10, 10)

# Assign the model to the device
model = model.to(device)

print(device)

In [ ]:

class BiLSTM(nn.Module):
    def __init__(self):
        super(BiLSTM, self).__init__()
        self.hidden_size = 64
        drp = 0.1
        n_classes = len(le.classes_)
        self.embedding = nn.Embedding(max_features, embed_size)
        self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, bidirectional=True, batch_first=True)
        self.linear = nn.Linear(self.hidden_size*4 , 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(64, n_classes)
    def forward(self, x):
        #rint(x.size())*
        x = x.long()
        h_embedding = self.embedding(x)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))*
        h_lstm, _ = self.lstm(h_embedding)
        avg_pool = torch.mean(h_lstm, 1)
        max_pool, _ = torch.max(h_lstm, 1)
        conc = torch.cat(( avg_pool, max_pool), 1)
        conc = self.relu(self.linear(conc))
        conc = self.dropout(conc)
        out = self.out(conc)
        return out



In [ ]:
import time
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder


# Define hyperparameters
n_epochs = 30
embed_size = 300
batch_size = 32
# Initialize your BiLSTM model
model = BiLSTM()
model = model.to(device)

loss_fn = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.001)
model.cuda()

# Create a device (CPU or GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load and preprocess your data (replace with your data loading code)

# Label encoding (replace with your label encoding code)
le = LabelEncoder()
train_y = le.fit_transform(train_y)
test_y = le.transform(test_y)

# Convert data to PyTorch tensors and move to GPU
x_train = torch.tensor(train_X, dtype=torch.float32).to(device)
y_train = torch.tensor(train_y, dtype=torch.long).to(device)
x_cv = torch.tensor(test_X, dtype=torch.float32).to(device)
y_cv = torch.tensor(test_y, dtype=torch.long).to(device)

# Create Torch datasets
train_dataset = TensorDataset(x_train, y_train)
valid_dataset = TensorDataset(x_cv, y_cv)

# Create Data Loaders
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)

# Lists to store training and validation losses
train_loss = []
valid_loss = []

# Training loop
for epoch in range(n_epochs):
    start_time = time.time()

    # Set model to train configuration
    model.train()
    avg_loss = 0.

    for i, (x_batch, y_batch) in enumerate(train_loader):
        # Predict/Forward Pass
        y_pred = model(x_batch)

        # Compute loss
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        avg_loss += loss.item() / len(train_loader)

    # Set model to validation configuration - Doesn't get trained here
    model.eval()
    avg_val_loss = 0.
    val_preds = np.zeros((len(x_cv), len(le.classes_)))

    for i, (x_batch, y_batch) in enumerate(valid_loader):
        y_pred = model(x_batch).detach()
        avg_val_loss += loss_fn(y_pred, y_batch).item() / len(valid_loader)

        # Keep/store predictions
        val_preds[i * batch_size:(i + 1) * batch_size] = F.softmax(y_pred, dim=1).cpu().numpy()

    # Check accuracy
    val_accuracy = sum(val_preds.argmax(axis=1) == test_y) / len(test_y)

    train_loss.append(avg_loss)
    valid_loss.append(avg_val_loss)

    elapsed_time = time.time() - start_time

    print('Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t val_acc={:.4f} \t time={:.2f}s'.format(
        epoch + 1, n_epochs, avg_loss, avg_val_loss, val_accuracy, elapsed_time))


Epoch 1/30 	 loss=31.5862 	 val_loss=23.8340 	 val_acc=0.7331 	 time=15.68s
Epoch 2/30 	 loss=22.6448 	 val_loss=22.0117 	 val_acc=0.7518 	 time=14.99s
Epoch 3/30 	 loss=20.4770 	 val_loss=21.4219 	 val_acc=0.7607 	 time=8.22s
Epoch 4/30 	 loss=18.7065 	 val_loss=21.7036 	 val_acc=0.7571 	 time=7.98s


In [ ]:
pip install scikit-plot


In [ ]:
import scikitplot as skplt
y_true = [le.classes_[x] for x in test_y]
y_pred = [le.classes_[x] for x in val_preds.argmax(axis=1)]
skplt.metrics.plot_confusion_matrix(
    y_true,
    y_pred,
    figsize=(12,12),x_tick_rotation=90)

In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming y_true and y_pred are defined as before
y_true = [le.classes_[x] for x in test_y]
y_pred = [le.classes_[x] for x in val_preds.argmax(axis=1)]

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Print the confusion matrix as a text representation
print("Confusion Matrix:")
print(conf_matrix)
